# 1. Required packages

## For the interactivity I'm using _plotly_ library and _Jupyter Widgets_. 
It allows to prototype dashboard in some way. Hence, for correct execution of the current notebook one has to install the following packages: 
   - [plotly](https://plot.ly/python/) + [cufflinks](https://plot.ly/ipython-notebooks/cufflinks/)
   - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets)

and activate the required extensions for Jupyter. Feel free to do it manually or run the following script.

In [ ]:
# %%bash
# pip install plotly
# pip install cufflinks
# pip install ipywidgets
# jupyter nbextension install --py --sys-prefix widgetsnbextension
# jupyter nbextension install --py --sys-prefix plotlywidget
# jupyter nbextension enable --py --sys-prefix widgetsnbextension
# jupyter nbextension enable --py --sys-prefix plotlywidget

To validate the correct installation and activation of the required packages please execute:

In [1]:
%%bash 
jupyter nbextension list

Known nbextensions:
  config dir: /Users/lex/.jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      plotlywidget/extension  enabled 
  config dir: /Users/lex/anaconda3/etc/jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      nb_conda/main  enabled 
    tree section
      nb_conda/tree  enabled 


      - Validating: OK
      - Validating: OK
      - Validating: OK
      - Validating: OK
      - Validating: OK


# 2. Data I/O
We assume that one has already obtained the embeddings for the researched financial statement network. 

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from NetEmbs import *

MODE = "SimulatedData"


if MODE == "SimulatedData":
    EMBS_PATH = "Simulation_versionMetaDiff_directionCOMBI_walks30_pressure30_window3/TFsteps100000batch64_emb32/"
    embs = pd.read_pickle(EMBS_PATH+"cache/Embeddings.pkl")
    print("Embeddings have been uploaded to memory!")
    d = upload_data("Simulation/FSN_Data.db", limit=None)
    d = prepare_data(d)
    print("Supported information has been uploaded to memory!")

if MODE == "RealData":
    import extras
    import analysis
    EMBS_PATH = "model/14080_2017_versionMetaDiff_directionCOMBI_walks31_pressure30_window3/TFsteps100000batch64_emb32/"
    embs = pd.read_pickle(EMBS_PATH+"cache/Embeddings.pkl")
        # //////// TODO UPLOAD your data HERE \\\\\\\\\\
#     d = analysis.analysis("14082_2017")
    d = extras.getData("14080_2017")
        # //////// END  \\\\\\\\\\
    # TODO pay attention for the split argument below!
    if "Value" in list(d):
        need_split = True
    else:
        need_split = False
    d = prepare_dataMarcel(d, split=need_split)

Embeddings have been uploaded to memory!
Original shape of DataFrame is  (61158, 4)
Deleted all NaNs and Strings values from 'Value' column:  (61158, 4)
Before merging FAs columns titles are:  ['ID', 'Name', 'FA_Name', 'Value', 'Debit', 'Credit']
After merging FAs columns titles are:  ['ID', 'FA_Name', 'Credit', 'Debit', 'Name', 'Value']  shape is  (61050, 6)
After normalization shape of DataFrame is  (60740, 8)
Final shape of DataFrame is  (52175, 9)
Supported information has been uploaded to memory!


In [4]:
embs.head(2)

,ID,Emb,GroundTruth
0,8,"[-0.19695708, -0.22684997, 0.115069434, 0.1744...",Sales 21 btw
1,9,"[0.15847915, 0.0964187, -0.09496842, -0.130400...",Cost of Sales


In [5]:
d.head(10)

,ID,Signature,FA_Name,Credit,Debit,Name,Value,from,amount
0,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_ilpe,0.000000,0.001441,NoisyRightFA_ilpe_6,0.678201,False,0.678201
1,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_jhzg,0.000000,0.003988,NoisyRightFA_jhzg_6,1.877578,False,1.877578
2,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_mard,0.000000,0.001110,NoisyRightFA_mard_6,0.522545,False,0.522545
3,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_nahl,0.000000,0.002286,NoisyRightFA_nahl_6,1.076345,False,1.076345
4,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_oyij,0.000000,0.004095,NoisyRightFA_oyij_6,1.927631,False,1.927631
5,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_wpkx,0.000000,0.001024,NoisyRightFA_wpkx_6,0.482056,False,0.482056
6,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_zhlc,0.000000,0.001364,NoisyRightFA_zhlc_6,0.641926,False,0.641926
7,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",Revenue,0.826287,0.000000,Revenue_6,-389.000000,True,389.000000
8,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",Tax,0.173713,0.000000,Tax_6,-81.780718,True,81.780718
9,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",TradeReceivables,0.000000,0.984693,TradeReceivables_6,463.574438,False,463.574438


# 3. Interactive visualization

### 3.1 Decrease the dim of embeddings for visualization purpose

In [6]:
import os
from sklearn.manifold import TSNE
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
def dim_reduction(df, n_dim=2, rand_state=1):
    if "Emb" in list(df):
        tsne = TSNE(random_state=rand_state)
        embdf = pd.DataFrame(list(map(np.ravel, df["Emb"])))
        embed_tsne = tsne.fit_transform(embdf)
        df["x"] = pd.Series(embed_tsne[:, 0])
        df["y"] = pd.Series(embed_tsne[:, 1])
        return df
    else:
        raise KeyError("No Embs column in the given DataFrame!")

### 3.2 Visualization

In [7]:
from __future__ import print_function
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
from ipywidgets import interactive, HBox, VBox, widgets

## Testing histograms

In [ ]:
ex1 = d.head(1000).merge(embs, on="ID")

In [ ]:
ex1 = ex1[(ex1.label==1) | (ex1.label==2)]

In [ ]:
gr = ex1.groupby(["FA_Name", "from"])
counts = gr.size().to_frame(name='counts')
all_stat = counts.join(gr.agg({'amount': sum, 'Debit': lambda x: list(x), 'Credit': lambda x: list(x)})
      .rename(columns={'amount': 'amount_sum', 'Debit': 'Debit_list', 'Credit': 'Credit_list'}))\
.reset_index()
all_stat.sort_values('amount_sum', ascending=False, inplace=True)

In [ ]:
all_stat.head(2)

In [ ]:
test_h = {"Left": [(x[0], x[2], x[3], x[5]) for x in all_stat[all_stat["from"]==True].values[:5]], 
"Right": [(x[0], x[2], x[3], x[4]) for x in all_stat[all_stat["from"]==False].values[:5]]}['Right']

In [ ]:

for item in test_h:
    if len(item[3])>3:
        sns.distplot( item[3] , label=item[0], kde=False, bins=10)
plt.legend(frameon=False)
plt.show()

### WordClouds function

In [8]:
from collections import Counter
from wordcloud import WordCloud
# Count most frequent FA names in the given DataFrame OR FA names with the highest amount
def findMostCommonFAs_v2(df, labels_column="label", words_column="FA_Name", amount_column="amount", sort_mode="freq", n_top=4, vis=False, folder=""):
    if labels_column not in list(df):
        raise KeyError(f"Given column name {labels_column} is not presented in the given DataFrame! Only allows: {list(df)}!")
    if "from" not in list(df):
        raise KeyError(f"Please ensure that column 'from' is presented in your DataFrame!")
    for name, group in df.groupby(labels_column):
        print("Current cluster label is ", name)
        gr = group.groupby([words_column, "from"])
        counts = gr.size().to_frame(name='counts')
        all_stat = counts.join(gr.agg({amount_column: sum, 'Debit': lambda x: list(x), 'Credit': lambda x: list(x)})
              .rename(columns={amount_column: 'amount_sum', 'Debit': 'Debit_list', 'Credit': 'Credit_list'}))\
        .reset_index()
        if sort_mode == "freq":
            all_stat.sort_values('counts', ascending=False, inplace=True)
        elif sort_mode == "amount":
            all_stat.sort_values('amount_sum', ascending=False, inplace=True)
#             Store all statistict for N_TOP values as dictionary for further visualization
        text = {"Left": [(x[0], x[2], x[3], x[5]) for x in all_stat[all_stat["from"]==True].values[:n_top]], 
                "Right": [(x[0], x[2], x[3], x[4]) for x in all_stat[all_stat["from"]==False].values[:n_top]]}
        if vis:
            i = 0
            fig, axes = plt.subplots(2,2)
        for key, data in text.items():
            if sort_mode == "freq":
#             Take the most frequent FA names
                to_vis = [(str(item[0]), item[1]) for item in data]
            elif sort_mode == "amount":
                to_vis = [(str(item[0]), item[2]) for item in data]
#             print(key, "--->", [item[:3] for item in data])
            if vis:
#                 WordClouds
                axes[0, i].set_title(key, size=24)
                wc = WordCloud(background_color="white", width=800, height=400, max_font_size=84, min_font_size=14, repeat=False, relative_scaling=0.8, max_words=100)
                if len(to_vis)>0:
                    wc.generate_from_frequencies(dict(to_vis))
                else:
                    continue
                axes[0, i].axis("off")
                axes[0, i].imshow(wc, interpolation="bilinear")
#                 Histograhm
                [sns.distplot( item[3] , label=item[0], kde=False, bins=5, ax=axes[1, i]) for item in data if len(item[3])>4]
                axes[1,i].legend(frameon=False, fontsize=14)
                axes[1,i].set_xlim((0,1.0))
                i+=1
        if vis:
            plt.tight_layout()
#             plt.savefig(folder + "img/WordClouds/" + str(name), dpi=140, pad_inches=0.01)
            plt.show()

In [ ]:
findMostCommonFAs_v2(ex1, vis=True, sort_mode="freq")

In [ ]:
from collections import Counter
from wordcloud import WordCloud
# Count most frequent FA names in the given DataFrame OR FA names with the highest amount
def findMostCommonFAs(df, labels_column="label", words_column="FA_Name", amount_column="amount", sort_mode="freq", n_top=2, vis=False, folder=""):
    for name, group in df.groupby(labels_column):
        print("Current cluster label is ", name)
        cur_data = group.groupby(words_column, as_index=False).aggregate({amount_column: sum})
        fa_amounts = dict(zip(cur_data[words_column].values, cur_data[amount_column].values))
        text = {"Left": [(item[0], item[1], fa_amounts[item[0]]) for item in Counter(group[group["from"] == True][words_column].values).items()], 
                "Right": [(item[0], item[1], fa_amounts[item[0]]) for item in Counter(group[group["from"] == False][words_column].values).items()]}
        if vis:
            i = 1
            fig = plt.figure()  
        for key, data in text.items():
            if sort_mode == "freq":
#             Take the most frequent FA names
                output = sorted(data, key=lambda d: -d[1])
                to_vis = [(str(item[0]), item[1]) for item in output]
            elif sort_mode == "amount":
                output = sorted(data, key=lambda d: -d[2])
                to_vis = [(str(item[0]), item[2]) for item in output]
            print(key, "--->", output[:n_top])
            if vis:
                ax = fig.add_subplot(1,2,i)
                i+=1
                ax.set_title(key)
                wc = WordCloud(background_color="white", width=800, height=400, max_font_size=84, min_font_size=14, repeat=False, relative_scaling=0.8, max_words=100)
                if len(to_vis)>0:
                    wc.generate_from_frequencies(dict(to_vis))
                else:
                    continue
                ax.axis("off")
                ax.imshow(wc, interpolation="bilinear")
        if vis:
            plt.tight_layout()
#             plt.savefig(folder + "img/WordClouds/" + str(name), dpi=140, pad_inches=0.01)
            plt.show()

In [ ]:
findMostCommonFAs(ex1, vis=True)

### Helper functions

In [10]:
# Transform Matplotlib colormap into plotly colorscale:
import itertools
def matplotlib_to_plotly(color_map="tab10", pl_entries=10):
    cmap = matplotlib.cm.get_cmap(color_map)
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

def getColors_Markers(keys, cm="tab10", n_color=10, markers = ["circle", "diamond", "square"]):
    keys = sorted(keys)
    color_map = dict(zip(keys, matplotlib_to_plotly(cm, n_color)*(len(keys)//n_color+1)))
    marker_map = dict(zip(keys, list(itertools.chain(*[[m]*n_color for m in markers]))*(len(keys)//(3*n_color)+1)))
    return color_map, marker_map

### Clustering here

In [11]:
N_CLS = 8
embs = dim_reduction(cl_Agglomerative(embs, N_CLS))

First row of Data: 
 [-0.19695708 -0.22684997  0.11506943  0.17442311  0.20106015 -0.13254054
  0.24126534  0.16323344  0.1949625   0.09444448  0.16678071  0.16417685
 -0.12703338  0.15944019 -0.24492544 -0.00580015 -0.07937735 -0.17278388
 -0.07006469  0.32693356 -0.23947953  0.04118197 -0.16536982  0.20513551
  0.17499836 -0.29295653  0.0757151   0.21822007  0.14713573 -0.05681241
 -0.17912321 -0.12949651]


In [12]:
d.head(3)

,ID,Signature,FA_Name,Credit,Debit,Name,Value,from,amount
0,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_ilpe,0.0,0.001441,NoisyRightFA_ilpe_6,0.678201,False,0.678201
1,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_jhzg,0.0,0.003988,NoisyRightFA_jhzg_6,1.877578,False,1.877578
2,8,"([('Revenue', 0.82629), ('Tax', 0.17371)], [('...",NoisyRightFA_mard,0.0,0.001110,NoisyRightFA_mard_6,0.522545,False,0.522545


In [13]:
embs.head(3)

,ID,Emb,GroundTruth,label,x,y
0,8,"[-0.19695708, -0.22684997, 0.115069434, 0.1744...",Sales 21 btw,2,-13.842628,-3.367294
1,9,"[0.15847915, 0.0964187, -0.09496842, -0.130400...",Cost of Sales,0,62.452831,0.478696
2,10,"[0.21436667, 0.20458017, 0.17234606, -0.244239...",Tax disbursement,0,-22.818893,52.704872


In [14]:
# TODO before visualization one has to use t-SNE!
# Label text
description = widgets.Label(
        value=''
    )
# WordCouds area
wordCloudsOutput = widgets.Output()
# Table with JournalEntries data
table_titles = ["ID", "FA_Name", "Credit", "Debit", "label"]

if MODE == "RealData":
    table_titles = ["ID", "FA_Name", "accountDesc", "Credit", "Debit", "label"]
    
t = go.FigureWidget([go.Table(
    header=dict(values=table_titles,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))],
                    layout = go.Layout(
                            title="Journal Entries",
                            autosize=True,
                            width=1000,
                            height=400))
# Scatter plot
N_COLORS = 10
WORD_CLOUD_LABEL = "FA_Name"
#LEGEND_TITLE = "GroundTruth"
LEGEND_TITLE = "label"
tmp_p_see = None
# For selection via multiple traces... stupid way.
indexes = []
tr_nums = 0


def interactiveScatter(df, df_info, legend_title="label"):
    """Create FigureWidget with the scatter plot for the given DataFrame"""
    scatter_data = list()
    cmap, mmap = getColors_Markers(keys=df[legend_title].unique(), n_color=N_COLORS)
    for name, group in df.groupby(legend_title):
        scatter_data.append(go.Scatter(x=group.x, y=group.y, mode='markers', name=name, 
                                    text = group.apply(lambda row: f"ID={row.ID},   GroundTruth={row.GroundTruth}", axis=1),
                                    customdata = group.index.to_list(),
                                    marker=dict(color=cmap[name][1], 
                                                symbol=mmap[name])))
    f = go.FigureWidget(data=scatter_data,
                       layout = go.Layout(
                           title=f"t-SNE visualisation with coloring based on {legend_title}",
        hovermode='closest',
        autosize=True,
        width=1000,
        height=700))
    
    def printSignature(trace, points, *args):
        if len(points.point_inds)>0:
            ids = trace.customdata[points.point_inds[0]]
            row = df.iloc[ids]
            description.value = f"ID={row.ID},   GroundTruth={row.GroundTruth}"
    def selectBP(trace, points, *args):
        if len(points.point_inds)>0:
            ids = trace.customdata[points.point_inds[0]]
            row = df.iloc[[ids]]
            chosen_bps = df_info.merge(row, on="ID")
            t.data[0].cells.values = [chosen_bps[col] for col in t.data[0].header.values]
    
    def filterRows(selected_ids):
        row = df.iloc[selected_ids]
        chosen_bps = df_info.merge(row.drop(["GroundTruth"], axis=1), on="ID")
        return chosen_bps
    def updateTable(chosen_bps):
        t.data[0].cells.values = [chosen_bps[col] for col in t.data[0].header.values]
    def showClouds(chosen_bps):
        wordCloudsOutput.clear_output()
        with wordCloudsOutput:
            findMostCommonFAs_v2(chosen_bps, LEGEND_TITLE, WORD_CLOUD_LABEL, sort_mode="freq", vis=True, n_top=4)
    scatters = f.data
    max_traces = len(scatters)
    def selectBPs(trace,points,selector):
        global indexes
        global tr_nums
#         print(f"For trace index={points.trace_index} tr_nums is {tr_nums}")
        if not points.point_inds:
            pass
        else:
            indexes.extend([trace.customdata[cur_point] for cur_point in points.point_inds])
        tr_nums = tr_nums+1
        if tr_nums==max_traces:
            selected_data = filterRows(indexes)
            updateTable(selected_data)
            showClouds(selected_data)
            indexes = []
            tr_nums = 0
    # Hover text: ID and GroundTruth
    for scatter in scatters:
        scatter.hoverinfo = 'text'
        scatter.on_hover(printSignature) 
        scatter.on_click(selectBP)
        scatter.on_selection(selectBPs)

    # Selection
    return f
# @interact(Coloring=['label', 'GroundTruth'])
# def update(Coloring="label"):
#     print(Coloring)
#     f_scatter = interactiveScatter(embs, Coloring)
#     return VBox([description, f_scatter])
f_scatter = interactiveScatter(embs, d, LEGEND_TITLE)
VBox([description, f_scatter, t, wordCloudsOutput])

    'data': [{'customdata': [1, 2, 3, ..., 6492, 6501, 6508],
 …

In [ ]:
import sklearn
sklearn.metrics.pairwise.cosine_similarity(embs[embs.ID==10].Emb.values[0].reshape(1,-1), 
                                           embs[embs.ID==970].Emb.values[0].reshape(1, -1))